# Code for image cropping and perspective transform
Taken from the developer Adrian Rosebrock and used as main source for OpenNoteScanner:

https://www.pyimagesearch.com/2014/09/01/build-kick-ass-mobile-document-scanner-just-5-minutes/

https://www.pyimagesearch.com/2014/08/25/4-point-opencv-getperspective-transform-example/

In [1]:
# import the necessary packages
#from pyimagesearch.transform import four_point_transform
#from skimage.filters import threshold_local
import numpy as np
import argparse
import cv2
import imutils
from shutil import copyfile

import os

### User selected input values
The first part needs user change, the second part can probably be kept with the default values

In [2]:
###### the following features need to be changed by the user to desired values

# file path to the input images, that will be cropped by the script
input_images_folder = '/Users/narsenov/Documents/synced_docs/i-sense/code_cropping_script/Val_AHRI_Test_images/'
# output folder where the intermediate cropped images will be stored
output_cropped_folder = '/Users/narsenov/Documents/synced_docs/i-sense/code_cropping_script/cropped_NEW/'
# output folder where the images will be stored in which the squares could not be successfully detected
output_uncropped_folder = '/Users/narsenov/Documents/synced_docs/i-sense/code_cropping_script/uncropped/'
# output folder where eventually the defined areas of interest from the images will be stored
output_AOI_folder = '/Users/narsenov/Documents/synced_docs/i-sense/code_cropping_script/AOI/'

# define the coordinates of your areas of interest, as height and width ratio in respect to the 4 squares.
# The coordinates will have the form of: 
# image_AOI = im[int(x0*height_im):int(x1*height_im), int(y0*width_im):int(y1*width_im)]
x0, x1 = 0.09, 0.89
y0, y1 = 0.56, 0.90

####### the features below this line can be probably kept with the given default values,
####### but the user may consider changing them if script performance is bad

# what type of image files do you want the script to opearate on
file_extensions = ['.jpg', '.png']

# the size of the kernel when threshholding is being performed
thresh_size = 15

# the height of the image will be scaled down to an image with this height, so that working can go faster 
height_working_image=1000.

# defining the range of blurness in which the image will be looped for the detection of shapes
GBlur_bottom_value=7
GBlur_top_value=21
GBlur_interval=2

# tolerance range of what to consider a square, defining the minimal and maxim al ratio between the width and height of the object
hw_ratio_min = 0.85
hw_ratio_max = 1.15

### Functions below are taken from Adrian Rosenbrock:
https://www.pyimagesearch.com/2014/08/25/4-point-opencv-getperspective-transform-example/

In [3]:
def order_points(pts):
	# initialzie a list of coordinates that will be ordered
	# such that the first entry in the list is the top-left,
	# the second entry is the top-right, the third is the
	# bottom-right, and the fourth is the bottom-left
	rect = np.zeros((4, 2), dtype = "float32")
 
	# the top-left point will have the smallest sum, whereas
	# the bottom-right point will have the largest sum
	s = pts.sum(axis = 1)
	rect[0] = pts[np.argmin(s)]
	rect[2] = pts[np.argmax(s)]
 
	# now, compute the difference between the points, the
	# top-right point will have the smallest difference,
	# whereas the bottom-left will have the largest difference
	diff = np.diff(pts, axis = 1)
	rect[1] = pts[np.argmin(diff)]
	rect[3] = pts[np.argmax(diff)]
 
	# return the ordered coordinates
	return rect

def four_point_transform(image, pts):
	# obtain a consistent order of the points and unpack them
	# individually
	rect = order_points(pts)
	(tl, tr, br, bl) = rect
 
	# compute the width of the new image, which will be the
	# maximum distance between bottom-right and bottom-left
	# x-coordiates or the top-right and top-left x-coordinates
	widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
	widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
	maxWidth = max(int(widthA), int(widthB))
 
	# compute the height of the new image, which will be the
	# maximum distance between the top-right and bottom-right
	# y-coordinates or the top-left and bottom-left y-coordinates
	heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
	heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
	maxHeight = max(int(heightA), int(heightB))
 
	# now that we have the dimensions of the new image, construct
	# the set of destination points to obtain a "birds eye view",
	# (i.e. top-down view) of the image, again specifying points
	# in the top-left, top-right, bottom-right, and bottom-left
	# order
	dst = np.array([
		[0, 0],
		[maxWidth - 1, 0],
		[maxWidth - 1, maxHeight - 1],
		[0, maxHeight - 1]], dtype = "float32")
 
	# compute the perspective transform matrix and then apply it
	M = cv2.getPerspectiveTransform(rect, dst)
	warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
 
	# return the warped image
	return warped

In [ ]:
def width_calc(quader):
    '''Function that assumes quader is rectanlge and returns its width'''
    quad = np.resize(quader, (4,2))
    distances = []
    for coo in quad[1:]:
        d = np.sqrt((quad[0][0]-coo[0])**2 + (quad[0][1]-coo[1])**2)
        distances.append(d)

    return(min(distances))

Creating here a main function that finds the 4 squares in the corners and uses them. Problem with this technique:
- a fewer number of detected large test, because sometimes squares are outside of the image (~5% of images)
- the cropped part around the smaller device are almost unusable, because it is not fixed in the slot, but moves around
    - the problem here can be avoided by second iteration of square detection inside the slot of the small device

In [4]:
def crop_around_squares(input_images_folder, output_cropped_folder, output_uncropped_folder, file_extensions, 
         height_working_image, 
         GBlur_bottom_value, GBlur_top_value, GBlur_interval):
    """Main function, contains the loop and all the sub-functions in it.
    DON'T forget to add an extra '/' at the end of input and output folder.
    area_ratio is the ratio (imaged area small device)/(imaged area large device).
    l_w_ratio_large_upper and l_w_ratio_large_lower are the range margins for the ratio l/w between the lenght and width of the imaged large device."""
    
    # create a list of all the image-paths in a given folder
    file_names = [fn for fn in sorted(os.listdir(input_images_folder))
                  if any(fn.endswith(ext) for ext in file_extensions)]
    paths = [input_images_folder+file_name for file_name in file_names]
    
    flag_L = 'stay in loop'
    flag_S = 'stay in loop'
    i = 0
                
    # loop through all of the image-paths
    for im_path in paths[:]:
        path_end = os.path.basename(im_path)
        print('loop:', i)
        
        flag_L = 'stay in loop'
        flag_S = 'stay in loop'
        i+=1
        
        # loop through all the possible sizes of the Gaussian-Blur-kernel
        for GBlur_size in range(GBlur_bottom_value, GBlur_top_value, GBlur_interval):
            if flag_L == 'move to next image':
                continue


            # load the image and compute the ratio of the old height
            # to the new height, clone it, and resize it
            #image = cv2.imread(args["image"])
            image = cv2.imread(im_path)
            ratio = image.shape[0] /height_working_image
            orig = image.copy()
            image = imutils.resize(image, height = int(height_working_image))


            # convert the image to grayscale, blur it, and find edges
            # in the image
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (GBlur_size, GBlur_size), 0)
            # line below does threshholding in black and white
            gray = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, thresh_size,2)

            ########
            # Calculate the canny-edged image based on the median value of image
            v = np.median(gray)
            sigma = 0.33

            # apply automatic Canny edge detection using the computed median
            lower = int(max(0, (1.0 - sigma) * v))
            upper = int(min(255, (1.0 + sigma) * v))
            edged = cv2.Canny(gray, lower, upper)
            ########

            # iterating over increasing and decreasing the width of the edges to get rid of gaps and noisy fine structure
            edged = cv2.dilate(edged, None, iterations=1)
            edged = cv2.erode(edged, None, iterations=1)

#             cv2.imshow("Edged", edged)
#             cv2.waitKey(0)

            # find the contours in the edged image, keeping only the
            # largest ones, and initialize the screen contour
            cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
            cnts = imutils.grab_contours(cnts)
            cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:30]
            
#             cv2.drawContours(image, cnts, -1, (0, 255, 0), 2)
#             cv2.imshow("All edges", image)
#             cv2.waitKey(0)
              
            squares = []
            square_centers = []
            features = []
            for c in cnts:
                # approximate the contour
                peri = cv2.arcLength(c, True)
                approx = cv2.approxPolyDP(c, 0.02 * peri, True)
                
#                 cv2.drawContours(image, [approx], -1, (0, 255, 0), 2)
#                 cv2.imshow("Outline", image)
#                 cv2.waitKey(0)

                # if our approximated contour has four points, then we
                # can assume that we have found our screen
                if len(approx) == 4:
                    
                    (x, y, w, h) = cv2.boundingRect(approx)
                    ar = w / float(h)
                    
                    # we need first to order the points so that the first point is always the top left
                    approx = order_points(approx.reshape((4,2))).astype(int)
        
                    # define the four corner points of one pf the possible squares
                    A, B, C, D = approx[0], approx[1], approx[2], approx[3]        
                    x, y = A[0], A[1]
                    h, w = np.sqrt((B[1]-A[1])**2 + (B[0]-A[0])**2), np.sqrt((B[1]-C[1])**2 + (B[0]-C[0])**2)
                    hw_ratio = w / float(h)

                    # a square will have an aspect ratio that is approximately
                    # equal to one, otherwise, the shape is a rectangle.
                    # we also make sure that the rectangle is small
                    if hw_ratio >= hw_ratio_min and hw_ratio <= hw_ratio_max:
                        features.append([approx, ar, x, y, w, h])
                        
            # we are getting rid of outliers here
            features = np.array(features)
            for (approx, ar, x, y, w, h) in features:
                if (abs(w - np.median(features[:,4])) < 0.6*np.median(features[:,4])): # 1. * np.std(features[:,4])
                    
                        
                    # append the sauqres and centers of the squares into a list
                    square_centers.append([int(x+w/2), int(y+h/2)])
                    squares.append(approx)
            
            # crop the large image correspondingly,
            # but first check if 4 squares are detected af their centers also form a square
            
#             print("squares:", squares)
#             print('square centers:', square_centers)
            
#             cv2.drawContours(image, np.array(preliminary_squares), -1, (0, 255, 0), 2)
#             cv2.imshow("Preliminary Squares", image)
#             cv2.waitKey(0)

            try:
                # get rid of overlapping square centers
                square_centers_filtered = [square_centers[0]]
                for sq in square_centers:
                    flag = 'unique'
                    for squ in square_centers_filtered:
                        if np.sqrt((sq[0]-squ[0])**2 + (sq[1]-squ[1])**2) < 50:
                            flag = 'not unique'
                    if flag == 'unique':
                        square_centers_filtered.append(sq)

#                 print("square centers filtered:", square_centers_filtered)
            
                warped = four_point_transform(orig, np.array(square_centers_filtered).reshape(4, 2)*ratio)
                cv2.imwrite(output_cropped_folder+'NEW_'+path_end, warped)
                flag_L = 'move to next image'
            except:
                if GBlur_size == (GBlur_top_value-GBlur_interval):
                    copyfile(im_path, output_uncropped_folder+'NEW_'+path_end)

In [9]:
def crop_around_AOI(output_cropped_folder, output_AOI_folder, file_extensions):
    '''This function serves just to take the pictures, that have been cropped accordingly to the squares detected in them
    and then out of these to crop out the area of interest (AOI) specified by coordinates - these are the final images'''
    
    file_names = [fn for fn in sorted(os.listdir(output_cropped_folder))
                  if any(fn.endswith(ext) for ext in file_extensions)]
    paths = [output_cropped_folder+file_name for file_name in file_names]
    
    for im_path in paths[:]:
        path_end = os.path.basename(im_path)
        
        im = cv2.imread(im_path)
        h, w = im.shape[0], im.shape[1]
        # rotate the images, if the second dimension is larger
        if im.shape[1]>im.shape[0]:
            im = cv2.rotate(im, cv2.ROTATE_90_COUNTERCLOCKWISE)
        im_AOI = im[int(x0*h):int(x1*h), int(y0*w):int(y1*w)]
        cv2.imwrite(output_AOI_folder+'AOI'+path_end, im_AOI)

In [10]:
crop_around_squares(input_images_folder, output_cropped_folder, output_uncropped_folder,
     file_extensions, height_working_image,
     GBlur_bottom_value, GBlur_top_value, GBlur_interval)   

crop_around_AOI(output_cropped_folder, output_AOI_folder, file_extensions)

loop: 0
loop: 1


/Users/narsenov/anaconda3/envs/cv_py35/lib/python3.5/site-packages/ipykernel_launcher.py:103: RuntimeWarning: divide by zero encountered in double_scalars


loop: 2
loop: 3
loop: 4
loop: 5
loop: 6
loop: 7
loop: 8
loop: 9
loop: 10
loop: 11
loop: 12
loop: 13
loop: 14
loop: 15
loop: 16
loop: 17
loop: 18
loop: 19
loop: 20
loop: 21
loop: 22
loop: 23
loop: 24
loop: 25
loop: 26
loop: 27
loop: 28
loop: 29
loop: 30
loop: 31
loop: 32
loop: 33
loop: 34
loop: 35
loop: 36
loop: 37
loop: 38
loop: 39
loop: 40
loop: 41
loop: 42
loop: 43
loop: 44
loop: 45
loop: 46
loop: 47
loop: 48
loop: 49
loop: 50
loop: 51
loop: 52
loop: 53
loop: 54
loop: 55
loop: 56
loop: 57
loop: 58
loop: 59
loop: 60
loop: 61
loop: 62
loop: 63
loop: 64
loop: 65
loop: 66
loop: 67
loop: 68
loop: 69
loop: 70
loop: 71
loop: 72
loop: 73
loop: 74
loop: 75
loop: 76
loop: 77
loop: 78
loop: 79
loop: 80
loop: 81
loop: 82
loop: 83
loop: 84
loop: 85
loop: 86
loop: 87
loop: 88
loop: 89
loop: 90
loop: 91
loop: 92
loop: 93
loop: 94
loop: 95
loop: 96
loop: 97
loop: 98
loop: 99


# When to finish the code:
- provide as input the coordinates of the object you are interested in
- ask Jobie if he is the background author? Ask for permission to use the background
- upload the background also on github

## Publish 3 different codes:
- Detecting just the largest quader shape
- detecting a shape with given side ratio and approximate area ratio
- detecting the 4 squares in the background and than cropping around a specified area